In [1]:
# import libraries 
# ----------------
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

In [150]:
# check .mat file 
# ---------------
from scipy.io import whosmat
whosmat('OBC_synthetic_data_UD.mat') # to list variables inside mat file

[('vx', (52000, 2400), 'double'),
 ('vz', (52000, 2400), 'double'),
 ('pressure', (52000, 2400), 'double'),
 ('vx_inj', (52000, 2400), 'double'),
 ('vz_inj', (52000, 2400), 'double'),
 ('pressure_inj', (52000, 2400), 'double'),
 ('vx_up', (52000, 2400), 'double'),
 ('vz_up', (52000, 2400), 'double')]

In [37]:
# load .mat file 
# --------------
from scipy.io import loadmat
mat = loadmat('OBC_synthetic_data_UD.mat')  # to load mat file
# convert them to numpy array
vx = mat['vx']
vz = mat['vz']
vx_up = mat['vx_up']
vz_up = mat['vz_up']
vx_inj = mat['vx_inj']
vz_inj = mat['vz_inj']
pressure     = mat['pressure']
pressure_inj = mat['pressure_inj']

In [42]:
# initialize parameter 
# --------------------
nx = vx.shape[1] # number of cells in x direction
dx = 0.25        # cell size
dt = 5e-6        # time step
alpha = 1800     # P-wave velocity
beta = 700       # S-wave velocity
rho = 1600       # density

In [ ]:
# calculate wavefield gradients 
# -----------------------------
from sympy.calculus import finite_diff_weights # see http://docs.sympy.org/latest/modules/calculus/index.html
c = finite_diff_weights(1, [-3/2, -1/2, 1/2, 3/2], 0) # FD coefficients according to Fornberg (1988)
c = c[-1][-1] # FD weights for 1st derivative

# allocate zero gradients
gradx_vx = np.zeros((vx.shape[0],nx))
gradx_vz = np.zeros((vx.shape[0],nx))

for n in range(2,(vx.shape[1])-2):
    # horizontal gradients of vx
    gradx_vx[:,n] = ((c[0]*vx[:,n-1]) + (c[1]*vx[:,n])   + (c[2]*vx[:,n+1]) + (c[3]*vx[:,n+2])) / dx
    # horizontal gradients of vz
    gradx_vz[:,n] = ((c[0]*vz[:,n-2]) + (c[1]*vz[:,n-1]) + (c[2]*vz[:,n])   + (c[3]*vz[:,n+1])) / dx

# up/down separation 
# ------------------
# upgoing vx
vx_up       = (1/2.) * (vx + ((alpha*(1 - ((2*beta)/alpha))) * (gradx_vz.cumsum(axis=0)*dt))) 
# upgoing vz
vz_up       = (1/2.) * (vz - (( beta*(1 - ((2*beta)/alpha))) * (gradx_vx.cumsum(axis=0)*dt)) - ((1/(rho*alpha))*pressure)) 
# upgoing pressure
pressure_up = (1/2.) * (pressure - ((alpha * rho) * vz)) 

In [ ]:
# plotting configuration 
# ----------------------
figure()

# recorded vz
subplot(2,3,1)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vz); caxis([-1 1]/1e8)
xlabel('Distance (m)', 'Fontsize',20);
set(gca,'XTick', [600 1200 1800])
set(gca,'XTickLabel',{'-150', '0', '150'});
ylabel('Time (s)', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 60000]);
set(gca,'YTickLabel',{'0.00', '0.05', '0.10', '0.15', '0.20', '0.25', '0.30'});
title('Recorded v_z', 'Fontsize',20)
colormap('cs_bluewhitered')
xlim([500 1900])

# upgoing vz - reference solution
subplot(2,3,2);
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vz_inj); caxis([-1 1]/1e8)
xlabel('Distance (m)', 'Fontsize',20);
set(gca,'XTick', [600 1200 1800])
set(gca,'XTickLabel',{'-150', '0', '150'});
set(gca,'YTick', [10000 15000 20000 25000 30000]);
set(gca,'YTickLabel',{'0.050', '0.075', '0.100', '0.125', '0.150'});
title('Full vertical wavefield at 50 m', 'Fontsize',20)
title('Upoing v_z - reference solution', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 60000]);
set(gca,'YTickLabel',{'0.00', '0.05', '0.10', '0.15', '0.20', '0.25', '0.30'});
colormap(cs_bluewhitered)
xlim([500 1900])

# upgoing vz - filters
subplot(2,3,3);
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vz_up); caxis([-1 1]/1e8)
set(gca,'XTick', [600 1200 1800])
set(gca,'XTickLabel',{'-150', '0', '150'});
xlabel('Distance (m)', 'Fontsize',20);
title('Upgoing v_z - filters', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 60000]);
set(gca,'YTickLabel',{'0.00', '0.05', '0.10', '0.15', '0.20', '0.25', '0.30'});
colormap(cs_bluewhitered)
xlim([500 1900])

plt.show()